# Hypersonic TPS Simulator #

In [16]:
import numpy as np
import json
import physics
from solvers import *
from solvers.fd_mol_rhs import *
from solvers.solve_ivp_integrator import *
import visualization
import h5py
import math
import time

In [17]:
def save_results_to_h5(filename, t_sol, T_sol, L_sol, params):
    """
    Save simulation results and parameters into an HDF5 file.
    
    Args:
        filename: str, path to output .h5 file
        t_sol: 1D numpy array, time points
        T_sol: 2D numpy array, temperature profile over time
        L_sol: 1D numpy array, ablation depth
        params: dict, simulation parameters
    """
    with h5py.File(filename, "w") as f:
        # Save datasets
        f.create_dataset("time", data=t_sol)
        f.create_dataset("temperature", data=T_sol)
        f.create_dataset("ablation_depth", data=L_sol)
        
        # Save parameters as attributes
        param_group = f.create_group("parameters")
        for key, value in params.items():
            # HDF5 attributes cannot store dicts directly, store as string if needed
            if isinstance(value, (int, float, str)):
                param_group.attrs[key] = value
            elif isinstance(value, np.ndarray):
                param_group.create_dataset(key, data=value)
            else:
                param_group.attrs[key] = str(value)

    print(f"Results successfully saved to {filename}")

In [18]:
def driver(mach, altitude, t_max, material_id):
    """
    Driver function to run the hypersonic TPS simulation.

    Parameters:
        mach: Free-stream Mach number
        altitude: Altitude in KILOMETERS
        t_max: Maximum simulation time in seconds
        material_id: Material ID for TPS material (MUST BE ONE INDEXED)

    Returns:
        results: Dictionary containing time, temperature profile, and TPS thickness over time
    """

    # Load parameters from JSON file
    with open('config/params.json', 'r') as f:
        params = json.load(f)

    with open('config/materials.json', 'r') as f:
        materials = json.load(f)

    physics.fill_all_params(mach, altitude, t_max, material_id - 1, params, materials)

    # Ensure solvers.fd_mol_rhs and solve_ivp_integrator are imported correctly
    t_sol, T_sol, L_sol = solve_ivp_integrator(fd_mol_rhs, params, strategy='Radau')

    print(f"Simulation complete.")
    timestamp = time.time()
    filename = f"data/sim{timestamp}-M{mach}_alt{altitude}_mat{params['material_name']}_tmax{t_max}s.h5"
    save_results_to_h5(filename, t_sol, T_sol, L_sol, params)
    
    

In [19]:
with open('config/materials.json', 'r') as f:
    materials = json.load(f)

physics.print_material_selection(materials)

material_id = ""

while True:
    material_id = input(f"Enter material ID from the above list (1-{len(materials)}): ").strip()
    try:
        material_id = int(material_id)
    except ValueError:
        print("Invalid input. Please enter a number.")
        continue
    if 0 < material_id <= len(materials):
        break
    else:
        print("Invalid material ID. Enter material ID from the above list (1-{len(materials)}).")



MATERIAL SELECTION:
1. Carbon-Carbon (k=80.0 W/m·K, rho=1600.0 kg/m³, cp=1050.0 J/kg·K, T_abl=2200 K)
2. C/SiC Composite (k=45.0 W/m·K, rho=2500.0 kg/m³, cp=900.0 J/kg·K, T_abl=1900 K)
3. C/C-SiC Composite (k=55.0 W/m·K, rho=2200.0 kg/m³, cp=950.0 J/kg·K, T_abl=2100 K)
4. HfC UHTC (k=25.0 W/m·K, rho=12600.0 kg/m³, cp=280.0 J/kg·K, T_abl=4200 K)
5. TaC UHTC (k=22.0 W/m·K, rho=14500.0 kg/m³, cp=265.0 J/kg·K, T_abl=4100 K)
6. ZrC UHTC (k=20.0 W/m·K, rho=6600.0 kg/m³, cp=300.0 J/kg·K, T_abl=3825 K)
7. NbC UHTC (k=14.0 W/m·K, rho=7800.0 kg/m³, cp=320.0 J/kg·K, T_abl=3873 K)
8. AVCOAT (k=0.5 W/m·K, rho=500.0 kg/m³, cp=1500.0 J/kg·K, T_abl=1200 K)
9. PICA-X (k=0.09 W/m·K, rho=280.0 kg/m³, cp=1200.0 J/kg·K, T_abl=900 K)
10. SIRCA (k=0.3 W/m·K, rho=320.0 kg/m³, cp=1300.0 J/kg·K, T_abl=811 K)


In [20]:
input_tuple = (25, 20, 60, material_id)

driver(*input_tuple)

Simulation complete.
Results successfully saved to data/sim1758974745.3883865-M25_alt20_matHfC UHTC_tmax60s.h5


In [ ]:
import h5py
import numpy as np

# Traverse data folder.
filename = ""

with h5py.File(filename, "r") as f:
    # Read datasets
    t_sol = f["time"][:]               # 1D array
    T_sol = f["temperature"][:]        # 2D array [time, space]
    L_sol = f["ablation_depth"][:]     # 1D array
    
    # Read parameters if needed
    params = {key: f["parameters"].attrs[key] for key in f["parameters"].attrs}

print("Time shape:", t_sol.shape)
print("Temperature shape:", T_sol.shape)
print("Ablation depth shape:", L_sol.shape)


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'data/sim1695841234-M8_alt30_mat_HfC_UHTC_tmax60s.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import matplotlib.pyplot as plt

# Plot the ablation depth over time
plt.plot(t_sol, L_sol)
plt.xlabel("Time [s]")
plt.ylabel("TPS thickness [m]")
plt.title("Ablation over Time")
plt.show()

# Plot temperature at the nose (first node) over time
plt.plot(t_sol, T_sol[:, 0])
plt.xlabel("Time [s]")
plt.ylabel("Temperature [K]")
plt.title("Nose Temperature")
plt.show()
